In [1]:
!pip install opencv-python mediapipe numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to extract keypoints from a frame
def extract_keypoints(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    if results.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
    else:
        return None

# Folder containing videos
videos_folder = '/content/drive/MyDrive/rope_skipping/raw videos'

# Iterate through videos in the folder
keypoints_data = []

for filename in os.listdir(videos_folder):
    if filename.endswith('.mp4'):  # Assuming all videos are in mp4 format
        video_path = os.path.join(videos_folder, filename)
        cap = cv2.VideoCapture(video_path)
        keypoints_list = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            keypoints = extract_keypoints(frame)
            if keypoints:
                keypoints_list.append(keypoints)

        cap.release()

        # Convert keypoints list to numpy array for each video
        keypoints_array = np.array(keypoints_list)

        # Calculate standard deviation across keypoints for each video
        keypoints_std = np.std(keypoints_array, axis=0)

        # Identify common keypoints based on a threshold (e.g., std < 0.1)
        common_keypoints_indices = np.where(np.std(keypoints_array, axis=0) < 0.1)[0]

        # Print common keypoints indices for each video
        print(f"Common keypoints indices for {filename}:", common_keypoints_indices)

        # Append common keypoints indices to keypoints_data
        keypoints_data.append(common_keypoints_indices)

# Release MediaPipe Pose model
pose.close()